# Lab 1: Problem 1 (MDP algorithms: tabular Q-learning)

*OpenAI gym FrozenLake environment*

Winter is here. You and your friends were tossing around a frisbee at the park
    when you made a wild throw that left the frisbee out in the middle of the lake.
    The water is mostly frozen, but there are a few holes where the ice has melted.
    If you step into one of those holes, you'll fall into the freezing water.
    At this time, there's an international frisbee shortage, so it's absolutely imperative that
    you navigate across the lake and retrieve the disc.
    However, the ice is slippery, so you won't always move in the direction you intend.
    The surface is described using a grid like the following

        SFFF
        FHFH
        FFFH
        HFFG

    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located

    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, and zero otherwise.
    
FrozenLake-v1 defines "solving" as getting average reward of 0.78 over 100 consecutive trials.

More documentation: https://www.gymlibrary.dev/environments/toy_text/frozen_lake/


In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from kret_studies import *
from kret_studies.notebook import *
from kret_studies.complex import *

logger = get_notebook_logger()

Loaded environment variables from /Users/Akseldkw/Desktop/Columbia/ORCS4529/.env.
/Users/Akseldkw/coding/kretsinger/data/nb_log.log


In [3]:
env_vars = os.environ.copy()
wand_db_dir = env_vars["OUTPUT_DIR"] + "/wandb"

In [4]:
# wandb set up for logging runs online and moving them to the leaderboard
# create a wandb account when prompted, or simply sign in if you already have an account
# !pip install wandb -qqq
import wandb

wandb.login()
run = wandb.init(dir=wand_db_dir)

wandb: Currently logged in as: akseldkw (akseldkw07) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
## DO NOT CHANGE THIS CELL
import numpy as np
import gymnasium as gym
from gymnasium.envs.toy_text.frozen_lake import FrozenLakeEnv

env = typing.cast(FrozenLakeEnv, gym.make("FrozenLake-v1"))
# env.seed(0)

## Cartoon Rendering

![Frozen-Lake](./Frozen-Gym.png)

For proper accounting rewards while you learn, we build a wrapper around env.step() and env.reset(). In an episode, every time you take an action the reward will be appended to the reward of the episode, and when ever the environment is reset (at the end of an epsiode), the episode reward is reset to 0.

In [6]:
from functools import wraps

## DO NOT CHANGE THIS CELL
# wrapper for accounting rewards
rEpisode = 0
rList = []
fixedWindow = 100
movingAverage = 0


def reset_decorate(func):
    @wraps(func)
    def func_wrapper(*args, **kwargs):
        global rList
        global movingAverage
        global rEpisode
        global fixedWindow
        rList.append(rEpisode)
        if len(rList) >= fixedWindow:
            movingAverage = np.mean(rList[len(rList) - fixedWindow : len(rList) - 1])
        rEpisode = 0
        return func(*args, **kwargs)

    return func_wrapper


env.reset = reset_decorate(env.reset)


def step_decorate(func):
    @wraps(func)
    def func_wrapper(*args, **kwargs):
        global rEpisode
        # Call the original step function and unpack the result
        result = func(*args, **kwargs)
        if len(result) == 5:
            s1, r, d, other, info = result
            rEpisode += r
            return (s1, r, d, other, info)
        else:
            raise ValueError("Unexpected number of return values from env.step")

    return func_wrapper


env.step = step_decorate(env.step)


def init():
    global rEpisode, rList, movingAverage
    rEpisode = 0
    rList = []
    movingAverage = 0
    return

Below we illustrate the execution of the Open AI gym enviornment using the policy of chosing random action in every state. Every time an action is taken the enviorment returns a tuple containing next state, reward, and the status (whether terminal state is reached or not).

In [7]:
### RANDOM SAMPLING EXAMPLE
num_episodes = 1000
# number of episodes you want to try
episode_max_length = 100
# you can explicitly end the epsiode before terminal state is reached

env.reset()
# env.render()
# execute in episodes
for i in range(num_episodes):

    # reset the environment at the beginning of an episode
    s = env.reset()
    d = False  # not done

    for t in range(episode_max_length):

        ################ Random action policy ###########################
        # play random action
        a = env.action_space.sample()
        # get new state, reward, done
        s, r, d, _, info = env.step(a)
        #################################################################

        # break if done, reached terminal state
        if d == True:
            break

    # log per-episode reward and moving average over 100 episodes
    wandb.log(
        {
            "random reward": rEpisode,
            "random reward moving average": movingAverage,
            "random episode": i,
        }
    )

Implement tabular Q-learning (*YOU SHOULD ONLY CHANGE THE CELL BELOW*)

In [8]:
# def render_safe(mode:typing.Literal['human', 'rgb_array', 'ansii'] = 'human'):
#     env.render_mode = mode
#     env.render()
#     env.render_mode = None
# render_safe()

In [ ]:
"""## Action Space
The action shape is `(1,)` in the range `{0, 3}` indicating
which direction to move the player.

- 0: Move left
- 1: Move down
- 2: Move right
- 3: Move up

## Observation Space
The observation is a value representing the player's current position as
`current_row * ncols + current_col` (where both the row and col start at 0).
Therefore, the observation is returned as an integer.

For example, the goal position in the 4x4 map can be calculated as follows: 3 * 4 + 3 = 15.
The number of possible observations is dependent on the size of the map.
"""

In [ ]:
# initialize episodic structure

init()
num_episodes = 1
# number of training episodes, you can increase this to train more
episode_max_length = 100
# initialize discount factor, learning rate
gamma = 0.95
learnRate = 0.8

env.observation_space = typing.cast(gym.spaces.Discrete, env.observation_space)
env.action_space = typing.cast(gym.spaces.Discrete, env.action_space)
grid_env = typing.cast(FrozenLakeEnv, env.unwrapped)
nrows, ncols = grid_env.nrow, grid_env.ncol
# env.env = typing.cast(FrozenLakeEnv, env.env)
# create Q table
Q = np.zeros(
    [env.observation_space.n, env.action_space.n]
)  # Q(s,a). The Q-values from this array will be used to evaluate your policy.
n = np.zeros(
    [env.observation_space.n, env.action_space.n]
)  # recording number of trails for each arm

def available_actions(state: int) -> list[int]:
    """Return action indices that keep the agent inside the board."""
    row, col = divmod(state, ncols)  # state == row * ncols + col
    actions: list[int] = []
    if col > 0:
        actions.append(0)  # left
    if row < nrows - 1:
        actions.append(1)  # down
    if col < ncols - 1:
        actions.append(2)  # right
    if row > 0:
        actions.append(3)  # up
    return actions

# execute in episodes
for i in range(num_episodes):
    print("Episode ", i)
    # reset the environment at the beginning of an episode
    reset_state = env.reset()
    s = reset_state[0]
    terminated = False  # not done

    for t in range(episode_max_length):

        ###########SELCT ACTION a for state s using Q-values ##################
        valid_actions = available_actions(s)
        if not valid_actions:
            raise ValueError(f"No available actions for state {s}")
        # pick uniformly at random among the valid moves
        a = int(np.random.choice(valid_actions))
        print(f"State {s} Action {a}")
        # currently selecting a random action, change this TODO

        # get new state, reward, done
        state, reward, terminated, truncated, info = env.step(a)

        ##### update Q(s,a) ############
        if terminated:
            next_max = 0.0
        else:
            next_actions = available_actions(state)
            next_max = float(np.max(Q[state, next_actions])) if next_actions else 0.0
        td = float(reward) + gamma * next_max - Q[s, a]
        Q[s, a] = Q[s, a] + learnRate * td
        n[s, a] += 1

        # break if done, reached terminal state
        if terminated == True:
            print(f"Terminated {i=} {t=}")
            break
        s = state

    # log per-episode reward and moving average over 100 episodes
    wandb.log(
        {
            "training reward": rEpisode,
            "training reward moving average": movingAverage,
            "training episode": i,
        }
    )
wandb.run = typing.cast(wandb.Run, wandb.run)
wandb.run.summary["number of training episodes"] = num_episodes


Episode  0
State 0 Action 2
State 0 Action 0
State 4 Action 2
Terminated i=0 t=2 


In [12]:
reset_state

(0, {'prob': 1})

In [10]:
info

{'prob': 0.33333333333333337}

In [ ]:
rList, rEpisode

([0], 0)

In [ ]:
dtt(Q, n, names=["Q-values", "Number of trials per (s,a)"])

,0,1,2,3
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0


In [ ]:
type(env.action_space)

gymnasium.spaces.discrete.Discrete

In [ ]:
%%wandb
## DO NOT CHANGE THIS CELL. CHANGING ANY PART OF THIS CELL CAN DISQUALIFY THE SUBMISSION
#Evaluation of trained policy
init()
num_episodes=1000; #number of episodes for evaluation
episode_max_length=100
movingAverageArray=[]
score=0
env.reset()
for i in range(num_episodes):
    s = env.reset()
    d = False #not done
    for t in range(episode_max_length):
        a = np.argmax(Q[s,:])
        s, r, d, _ = env.step(a)
        if d == True:
            break
    #log per-episode reward and moving average over 100 episodes
    wandb.log({ "evaluation reward" : rEpisode, "evaluation reward moving average" : movingAverage, "evaluation episode" : i})
    movingAverageArray.append(movingAverage)
    #score is x if there is a window of 100 consecutive episodes where moving average was at least x
    if i>100:
        score=max(score,min(movingAverageArray[i-100:i-1]))


wandb.run.summary["score"]=score

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
run.finish()

random episode,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇████
random reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁
random reward moving average,▁▁▁▁▁▃▅▅▃▃▃▃▃▃▃▁▁▁▁▅▅▆█▅▅▃▁▁▁▁█████▃▃▃▃▃
training episode,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇████
training reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training reward moving average,▃▃▃▁▁▁▃▃▆███▆▆▆▆▆▆▃▃▃▁▁▁▁▃▃▆▃▃▃▃▆▆▆▆▆▆▃▁
number of training episodes,1000
random episode,999
random reward,0
random reward moving average,0.0101
training episode,999
